## Lendo dados

In [3]:
import pandas as pd

cursos = pd.read_json('data/alura-cursos-formacoes-dataset.json')

In [5]:
cursos['description'] = cursos['description'].apply(lambda x: x['raw'])

cursos_df = cursos[['id','title','description','type','siteUrl']]

In [6]:
cursos_df

,id,title,description,type,siteUrl
0,DEGREE-828628,Flutter: cursos para automatizar testes,<p>Aprenda a usar e personalizar testes automa...,DEGREE,/formacao-cursos-testes-automatizados-aplicati...
1,DEGREE-909375,Dart: cursos para aprender a linguagem utiliza...,<p>Se você busca uma linguagem para desenvolve...,DEGREE,/formacao-dart-cursos-aprender-linguagem-utili...
2,DEGREE-907699,"CSS avançado: implementando designs com Grid, ...",<p>Se você já conhece <strong>HTML</strong> e ...,DEGREE,/formacao-css-cursos-transformar-designs-grid-...
3,DEGREE-824634,"Angular: construa um Design System com Nx, Mon...","<p>Nesta formação, você vai desenvolver do zer...",DEGREE,/formacao-angular-cursos-construir-design-syst...
4,DEGREE-515205,Certificação AZ-900: Microsoft Azure Fundamentals,<p>O curso <strong>AZ-900 - Microsoft Azure Fu...,DEGREE,/formacao-certificacao-az-900-microsoft-azure-...
...,...,...,...,...,...
2495,HIPSTERSTECH-448,Primeiros Passos em Data Science: Do Excel e B...,<p>Escutamos falar muito sobre data science ou...,HIPSTERSTECH,https://www.alura.com.br/podcast/primeiros-pas...
2496,DEVSEMFRONTEIRAS-9178,"Engenheiro de Software PHP em Helsinque, Finlâ...",<p>O mossoroense Cid Lopes se familiarizou com...,DEVSEMFRONTEIRAS,https://www.alura.com.br/podcast/engenheiro-de...
2497,HIPSTERSTECH-9157,Histórias de Black Friday na Tecnologia – Hips...,<p>Hoje o papo é sobre Black Friday!</p>\n<p>N...,HIPSTERSTECH,https://www.alura.com.br/podcast/historias-de-...
2498,HIPSTERSTECH-9241,Case Livelo: Order Management System – Hipster...,<p>Hoje o papo é sobre OMS!</p>\n<p>Neste epis...,HIPSTERSTECH,https://www.alura.com.br/podcast/case-livelo-o...


## Começando com CRUD

### Criando cliente

In [7]:
password='XxZiSgIG'

from elasticsearch import Elasticsearch


# Configurações do Elasticsearch
ELASTICSEARCH_HOST = "http://localhost:9200/"
ELASTICSEARCH_USER = "elastic"  # Usuário padrão
ELASTICSEARCH_PASSWORD = password  # senha 

# Conectar ao Elasticsearch
es = Elasticsearch(
    [ELASTICSEARCH_HOST],
    http_auth=(ELASTICSEARCH_USER, ELASTICSEARCH_PASSWORD),
    verify_certs=False  # Desabilitar a verificação do SSL
)

# Verificar a conexão
if es.ping():
    print("Conexão bem-sucedida ao Elasticsearch!")
else:
    print("Falha ao conectar ao Elasticsearch. Verifique as configurações.")


Conexão bem-sucedida ao Elasticsearch!


/tmp/ipykernel_49650/2922222961.py:12: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


### Criando um Index

In [9]:
response = es.indices.create(index='alura-cursos')

print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'alura-cursos'}


### Deletando o index

In [8]:
response = es.indices.delete(index='alura-cursos')

### Inserindo Documentos

In [10]:
registro = dict(cursos_df.iloc[0])

In [11]:
registro

{'id': 'DEGREE-828628',
 'title': 'Flutter: cursos para automatizar testes',
 'description': '<p>Aprenda a usar e personalizar testes automatizados - a ferramenta do Flutter para garantir a qualidade do código, descubra como criar testes de unidade, de widgets, e de integração, a simular dependências externas usando o mockito e otimizar a cobertura de testes!</p>\r\n',
 'type': 'DEGREE',
 'siteUrl': '/formacao-cursos-testes-automatizados-aplicativos-flutter'}

In [12]:
create_response = es.index(index='alura-cursos', body=registro)

In [13]:
id_registro = create_response['_id']

In [14]:
create_response

ObjectApiResponse({'_index': 'alura-cursos', '_id': '6cPOEJYBf5GUU5vep9kl', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

### Atualizar um documento

In [15]:
registro['title'] = registro['title'].lower()

In [16]:
registro

{'id': 'DEGREE-828628',
 'title': 'flutter: cursos para automatizar testes',
 'description': '<p>Aprenda a usar e personalizar testes automatizados - a ferramenta do Flutter para garantir a qualidade do código, descubra como criar testes de unidade, de widgets, e de integração, a simular dependências externas usando o mockito e otimizar a cobertura de testes!</p>\r\n',
 'type': 'DEGREE',
 'siteUrl': '/formacao-cursos-testes-automatizados-aplicativos-flutter'}

In [17]:
update_response = es.update(index='alura-cursos', body={"doc": registro}, id=id_registro)

In [18]:
update_response

ObjectApiResponse({'_index': 'alura-cursos', '_id': '6cPOEJYBf5GUU5vep9kl', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

### Inserindo Vários documentos

In [19]:
for _doc in cursos_df.to_dict(orient='records'):
    es.index(index='alura-cursos', body=_doc)


KeyboardInterrupt



### Inserindo em lote

In [20]:
from elasticsearch import helpers

In [21]:
actions = [
    {
        "_index": "alura-cursos",
        "_source": curso
    }
    for curso in cursos_df.to_dict(orient='records')
]

In [22]:
actions[:2]

[{'_index': 'alura-cursos',
  '_source': {'id': 'DEGREE-828628',
   'title': 'Flutter: cursos para automatizar testes',
   'description': '<p>Aprenda a usar e personalizar testes automatizados - a ferramenta do Flutter para garantir a qualidade do código, descubra como criar testes de unidade, de widgets, e de integração, a simular dependências externas usando o mockito e otimizar a cobertura de testes!</p>\r\n',
   'type': 'DEGREE',
   'siteUrl': '/formacao-cursos-testes-automatizados-aplicativos-flutter'}},
 {'_index': 'alura-cursos',
  '_source': {'id': 'DEGREE-909375',
   'title': 'Dart: cursos para aprender a linguagem utilizada no Flutter',
   'description': '<p>Se você busca uma linguagem para desenvolver aplicações rápidas em qualquer plataforma, <strong>o Dart pode ser uma escolha excelente.</strong> </p>\r\n<p>Essa linguagem tem se destacado por sua produtividade no desenvolvimento multiplataforma, combinada com uma flexibilidade que permite você criar tanto aplicações web, q

In [23]:
response = helpers.bulk(es, actions)

In [24]:
response

(2500, [])

## Entendendo Mapping

### Gerando uma amostra

In [26]:
sample_cursos_id_inteiro = cursos_df.sample(100)
sample_cursos_id_str = cursos_df.sample(100)

In [28]:
sample_cursos_id_inteiro['id'] = sample_cursos_id_inteiro['id'].apply(lambda x: int(x.split('-')[-1]))

In [29]:
sample_cursos_id_inteiro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 1956 to 655
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           100 non-null    int64 
 1   title        100 non-null    object
 2   description  100 non-null    object
 3   type         100 non-null    object
 4   siteUrl      100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.7+ KB


### Criando Index

In [30]:
response = es.indices.create(index='alura-cursos-mapping')

print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'alura-cursos-mapping'}


### Indexando dados com diferentes formatos

##### Ids (Long) 

In [31]:
actions = [
    {
        "_index": "alura-cursos-mapping",
        "_source": curso
    }
    for curso in sample_cursos_id_inteiro.to_dict(orient='records')
]

In [32]:
helpers.bulk(es,actions=actions)

(100, [])

In [39]:
actions = [
    {
        "_index": "alura-cursos-schema",
        "_source": curso
    }
    for curso in sample_cursos_id_inteiro.to_dict(orient='records')
]

In [40]:
helpers.bulk(es,actions=actions)

(100, [])

##### Ids (text) 

In [33]:
actions = [
    {
        "_index": "alura-cursos-mapping",
        "_source": curso
    }
    for curso in sample_cursos_id_str.to_dict(orient='records')
]

In [34]:
helpers.bulk(es,actions=actions)

BulkIndexError: 100 document(s) failed to index.

### Criando Mapping

In [35]:
mapping = {
    "mappings": {
    "properties": {
      "description": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "id": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "siteUrl": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "title": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "type": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      }
    }
  }
}

In [36]:
es.indices.create(index="alura-cursos-schema",body=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'alura-cursos-schema'})

In [37]:
from pprint import pprint 

pprint(dict(es.indices.get(index='alura-cursos-schema')))

{'alura-cursos-schema': {'aliases': {},
                         'mappings': {'properties': {'description': {'fields': {'keyword': {'ignore_above': 256,
                                                                                            'type': 'keyword'}},
                                                                     'type': 'text'},
                                                     'id': {'fields': {'keyword': {'ignore_above': 256,
                                                                                   'type': 'keyword'}},
                                                            'type': 'text'},
                                                     'siteUrl': {'fields': {'keyword': {'ignore_above': 256,
                                                                                        'type': 'keyword'}},
                                                                 'type': 'text'},
                                                     'title': {

## Compreendendo melhor a Indexação

## Processo de Ingestão

![Texto Alternativo](images/data_processing_flow.png)

## Análise de Texto

![Texto Alternativo](images/analysis-chain.png)

## Exemplo de texto

![Texto Alternativo](images/Analyzer-example.png)